### 实现影像的裁剪、拼接
#### 1. 影像裁剪

In [18]:
from osgeo import gdal
import numpy as np


In [19]:
path_img = 'data/Section-2/s2_chenggong_20200411_6bands_20m.tif'


In [20]:
dset = gdal.Open(path_img)
geo_trans = dset.GetGeoTransform()          ### 地理转换参数
print('geotrans:', geo_trans)
x_min, y_max = geo_trans[0], geo_trans[3]   ### 左上角坐标
x_max = geo_trans[0]+geo_trans[1]*dset.RasterXSize
y_min = geo_trans[3]+geo_trans[5]*dset.RasterYSize
print('extent:', x_min, x_max, y_min, y_max)


geotrans: (874420.0, 20.0, 0.0, 2769600.0, 0.0, -20.0)
extent: 874420.0 904220.0 2737000.0 2769600.0


In [34]:
### 设定裁剪范围
x_min_subs, x_max_subs = 880003, 890009
y_min_subs, y_max_subs = 2742005, 2752003


#### 计算裁剪范围对应图像坐标


In [35]:
## x_start_img, y_start_img -> 874420.0, 2769600.0
## x_start_subs, y_start_subs -> 880003, 2760003
col_start_subs = (x_min_subs - geo_trans[0])/geo_trans[1]
row_start_subs = (y_max_subs - geo_trans[3])/geo_trans[5]
print(row_start_subs, col_start_subs)


479.85 529.15


In [36]:
### 裁剪影像起点行列号调整，及裁剪影像起点地理坐标调整
row_start_subs_update, col_start_subs_update = round(row_start_subs), round(col_start_subs)
print('new start point of the image coordinate:', row_start_subs_update, col_start_subs_update)
### 更新裁剪范围起点地理坐标
x_min_subs_update = geo_trans[0] + col_start_subs_update*geo_trans[1]
y_max_subs_update = geo_trans[3] + row_start_subs_update*geo_trans[5]
print(x_min_subs_update, y_max_subs_update)


new start point of the image coordinate: 480 529
885000.0 2760000.0


#### 计算裁剪影像尺寸

In [37]:
x_size_subs = (x_max_subs-x_min_subs_update)/geo_trans[1]
y_size_subs = -(y_max_subs_update-y_min_subs)/geo_trans[5]
print(x_size_subs, y_size_subs)


750.45 499.75


In [38]:
### 更新遥感影像尺寸
x_size_subs_update, y_size_subs_update = round(x_size_subs), round(y_size_subs)
print(x_size_subs_update, y_size_subs_update)


750 500


#### 更新裁剪影像范围右下角点地理坐标

In [39]:
x_max_subs_update = x_min_subs_update + x_size_subs_update*geo_trans[1]
y_min_subs_update = y_max_subs_update + y_size_subs_update*geo_trans[5]
print(x_max_subs_update, y_min_subs_update)



900000.0 2750000.0


In [40]:
extent_subs_update = [x_min_subs_update, x_max_subs_update, y_min_subs_update, y_max_subs_update]  #
print('new extent:', extent_subs_update)


new extent: [885000.0, 900000.0, 2750000.0, 2760000.0]


#### 裁剪影像地理转换参数确定

In [41]:
geotrans_subs = [x_min_subs_update, 20, 0, y_max_subs_update, 0, -20]
geotrans_subs


[885000.0, 20, 0, 2760000.0, 0, -20]

#### 影像裁剪

In [42]:
### 获取裁剪影像数组
img_array = dset.ReadAsArray()
print(img_array.shape)          ### （波段数，行，列) -> (波段数,y,x)


(6, 1630, 1490)


In [43]:
### 获取裁剪影像数组
### 设定x,y方向上裁剪影像和原始影像左上角像元个数差异为0，709，获得裁剪影像数组
img_array_subs = img_array[:, 
                           row_start_subs_update:row_start_subs_update+y_size_subs_update, 
                           col_start_subs_update:col_start_subs_update+x_size_subs_update]
print(img_array_subs.shape)


(6, 500, 750)


In [44]:
path_subs = 'data/Section-5/s2_chenggong_20200411_6bands_20m_subs.tif'
driver = gdal.GetDriverByName('GTiff')
dset_subs = driver.Create(path_subs, 
                           xsize = x_size_subs_update, 
                           ysize = y_size_subs_update, 
                           bands=dset.RasterCount, 
                           eType=gdal.GDT_Int16)
dset_subs.SetGeoTransform(geotrans_subs)
dset_subs.SetProjection(dset.GetProjection())
for i in range(dset.RasterCount):
    outband = dset_subs.GetRasterBand(i+1)       ### 获取波段1
    outband.WriteArray(img_array_subs[i])        ### 将np.array()数组写入波段1
dset_subs = None


##### 课堂练习：基于gdal库练习影像裁剪，裁剪范围为：
x_min_subs2, x_max_subs2 = 885003, 900009   
y_min_subs2, y_max_subs2 = 2750005, 2760003   
并将该裁剪影像重采样为23米分辨率。  

In [45]:
### 对subs2影像重采样，采样分辨率为23米
path_resam_subs2 = 'data/Section-5/s2_chenggong_20200411_6bands_20m_subs_2.tif'
path_resam_ = 'data/Section-5/s2_chenggong_20200411_6bands_20m_subs_2_resam.tif'
warp_resam = gdal.Warp(destNameOrDestDS=path_resam_, srcDSOrSrcDSTab=path_resam_subs2, xRes=23, yRes=23)  ### 利用gdal.Warp()进行投影转换
warp_resam = None   ### !!关闭文件


#### 2. 影像拼接

#### 将20m分辨率subs1和23m分辨率subs2影像进行拼接。


In [46]:
path_img_1 = 'data/Section-5/s2_chenggong_20200411_6bands_20m_subs.tif'
path_img_2 = 'data/Section-5/s2_chenggong_20200411_6bands_20m_subs_2_resam.tif'


In [48]:
dset_subs1 = gdal.Open(path_img_1)
dset_subs2 = gdal.Open(path_img_2)
geo_trans_subs1 = dset_subs1.GetGeoTransform()        ### 地理转换参数
geo_trans_subs2 = dset_subs2.GetGeoTransform()        ### 地理转换参数
print('subs1:', geo_trans_subs1)
print('subs2:', geo_trans_subs2)
x_min_subs1 = geo_trans_subs1[0]
y_max_subs1 = geo_trans_subs1[3]
x_min_subs2 = geo_trans_subs2[0]
y_max_subs2 = geo_trans_subs2[3]
print(x_min_subs1, y_max_subs1)
print(x_min_subs2, y_max_subs2)


subs1: (880000.0, 20.0, 0.0, 2752000.0, 0.0, -20.0)
subs2: (885000.0, 23.0, 0.0, 2760000.0, 0.0, -23.0)
880000.0 2752000.0
885000.0 2760000.0


In [50]:
x_max_subs1 = x_min_subs1 + dset_subs1.RasterXSize * geo_trans_subs1[1]
y_min_subs1 = y_max_subs1 + dset_subs1.RasterYSize * geo_trans_subs1[5]
print('extent_subs1:', x_min_subs1, x_max_subs1, y_min_subs1, y_max_subs1)
x_max_subs2 = x_min_subs2 + dset_subs2.RasterXSize * geo_trans_subs2[1]
y_min_subs2 = y_max_subs2 + dset_subs2.RasterYSize * geo_trans_subs2[5]
print('extent_subs2:', x_min_subs2, x_max_subs2, y_min_subs2, y_max_subs2)


extent_subs1: 880000.0 890000.0 2742000.0 2752000.0
extent_subs2: 885000.0 899996.0 2749995.0 2760000.0


In [51]:
x_max_mosaic = max(x_max_subs1, x_max_subs2)
x_min_mosaic = min(x_min_subs1, x_min_subs2)
y_max_mosaic = max(y_max_subs1, y_max_subs2)
y_min_mosaic = min(y_min_subs1, y_min_subs2)
extent_mosaic = [x_min_mosaic, x_max_mosaic, y_min_mosaic, y_min_mosaic]
print('拼接影像范围：', extent_mosaic)


拼接影像范围： [880000.0, 899996.0, 2742000.0, 2742000.0]


In [52]:
### 定义拼接影像分辨率（采用拼接影像subs1分辨率）
x_res_mosaic, y_res_mosaic = geo_trans_subs1[1], geo_trans_subs1[5]
print(x_res_mosaic, y_res_mosaic)


20.0 -20.0


In [63]:
### 将待拼接影像裁剪为所定义分辨率。
path_img_2_resam = 'data/Section-5/s2_chenggong_20200411_6bands_20m_subs_2_resam_.tif'
warp_resam = gdal.Warp(destNameOrDestDS=path_img_2_resam, srcDSOrSrcDSTab=path_img_2, xRes=20, yRes=20)  ### 利用gdal.Warp()进行投影转换


In [64]:
dset_subs2 = gdal.Open(path_img_2_resam)
dset_subs2.GetGeoTransform()


(885000.0, 20.0, 0.0, 2760000.0, 0.0, -20.0)

In [65]:
x_size_mosaic = (x_max_mosaic - x_min_mosaic)/x_res_mosaic
y_size_mosaic = -(y_max_mosaic - y_min_mosaic)/y_res_mosaic
print(x_size_mosaic, y_size_mosaic)


999.8 900.0


In [66]:
### 尺寸整数化及拼接影像空间范围更新
x_size_mosaic_update, y_size_mosaic_update = round(x_size_mosaic), round(y_size_mosaic)
print(x_size_mosaic_update, y_size_mosaic_update)
## 更新拼接范围（右下角点坐标）
x_max_mosaic_update = x_min_mosaic + x_res_mosaic*x_size_mosaic_update
y_min_mosaic_update = y_max_mosaic + y_res_mosaic*y_size_mosaic_update
print(x_max_mosaic_update, y_min_mosaic_update)


1000 900
900000.0 2742000.0


In [67]:
### 获得拼接影像地理转换参数
geotrans_mosaic = [x_min_mosaic, x_res_mosaic, 0, y_max_mosaic, 0, y_res_mosaic]
geotrans_mosaic


[880000.0, 20.0, 0, 2760000.0, 0, -20.0]

In [68]:
### 建立空数组
img_array_mosaic = np.zeros(shape=(dset_subs1.RasterCount, y_size_mosaic_update, x_size_mosaic_update))
img_array_mosaic.shape


(6, 900, 1000)

In [69]:
### 待拼接影像起点在拼接影像上位置
### subs1
row_start_subs1 = int((y_max_subs1 - y_max_mosaic)/y_res_mosaic)
col_start_subs1 = int((x_min_subs1 - x_min_mosaic)/x_res_mosaic)
print(row_start_subs1, col_start_subs1)
img_array_mosaic[:, row_start_subs1:row_start_subs1+dset_subs1.RasterYSize, \
                 col_start_subs1:col_start_subs1+dset_subs1.RasterXSize] = dset_subs1.ReadAsArray()

### subs2
row_start_subs2 = int((y_max_subs2 - y_max_mosaic)/y_res_mosaic)
col_start_subs2 = int((x_min_subs2 - x_min_mosaic)/x_res_mosaic)
print(row_start_subs2, col_start_subs2)
img_array_mosaic[:, row_start_subs2:row_start_subs2+dset_subs2.RasterYSize, \
                 col_start_subs2:col_start_subs2+dset_subs2.RasterXSize] = dset_subs2.ReadAsArray()


400 0
0 250


#### 写出拼接影像

In [70]:
path_mosaic = 'data/Section-5/s2_chenggong_20200411_6bands_20m_subs_mosaic.tif'
driver = gdal.GetDriverByName('GTiff')
dset_mosaic = driver.Create(path_mosaic, 
                            xsize = x_size_mosaic_update, 
                            ysize = y_size_mosaic_update, 
                            bands=dset_subs1.RasterCount, 
                            eType=gdal.GDT_Int16)
dset_mosaic.SetGeoTransform(geotrans_mosaic)
dset_mosaic.SetProjection(dset_subs1.GetProjection())

for i in range(dset_subs1.RasterCount):
    outband = dset_mosaic.GetRasterBand(i+1)       ### 获取波段1
    outband.WriteArray(img_array_mosaic[i])        ### 将np.array()数组写入波段1
dset_mosaic = None

